# Calculating ion solutions

## Configuration

Hydrogen only. Requires `dilute-lte` excitation and `nebular` ionization to provide the first approximation for the solver

In [2]:
from tardis.io.configuration.config_reader import Configuration

config = Configuration.from_yaml("../../../../../tardis/tardis/plasma/tests/data/plasma_base_test_config.yml")

config.model.abundances.He = 0
config.model.abundances.H = 1

config.plasma.excitation = "dilute-lte"
config.plasma.ionization = "nebular"

Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

## Atomic data

Requires photoionization cross section data

In [3]:
from copy import deepcopy

from tardis.io.atom_data import AtomData
from tardis.model.base import SimulationState

atom_data = AtomData.from_hdf('/tardis-regression-data/atom_data/nlte_atom_data/TestNLTE_He_Ti.h5')

atom_data2 = deepcopy(atom_data)

atom_data2.prepare_atom_data([1], "macroatom", [], [(1, 0)])

sim_state = SimulationState.from_config(config, atom_data=atom_data)

Number of density points larger than number of shells. Assuming inner point irrelevant


In [4]:
import astropy.units as u
import numpy as np

from tardis.plasma.electron_energy_distribution import (
    ThermalElectronEnergyDistribution,
)
from tardis.plasma.radiation_field import (
    DilutePlanckianRadiationField,
)

rad_field = DilutePlanckianRadiationField(np.ones(20) * 10000 * u.K, dilution_factor=np.ones(20) * 0.5)

electron_dist = ThermalElectronEnergyDistribution(0, np.ones(20) * 10000 * u.K, np.ones(20) * 2e9 * u.cm**-3)

# Create a plasma

In [5]:
from tardis.plasma.assembly.base import PlasmaSolverFactory

plasma_solver_factory = PlasmaSolverFactory(atom_data, config)
plasma_solver_factory.prepare_factory([1],"tardis.plasma.properties.property_collections" )
plasma = plasma_solver_factory.assemble(sim_state.elemental_number_density, rad_field, sim_state.time_explosion, electron_dist.number_density, link_t_rad_t_electron=1)

Zeta_data missing - replaced with 1s. Missing ions: []
Zeta_data missing - replaced with 1s. Missing ions: []


# Create LTE properties

In [6]:
from tardis.plasma.properties.ion_population import IonNumberDensity


class LTEIonNumberDensity(IonNumberDensity):
    outputs = ('lte_ion_number_density', )
    latex_name = ('N_{i,j}^*',)

    def calculate(self, phi_Te, lte_partition_function_Te, number_density, electron_densities, block_ids, ion_threshold):
        return self.calculate_with_n_electron(
            phi_Te, lte_partition_function_Te, number_density, electron_densities, block_ids, ion_threshold)

In [7]:
from tardis.plasma.properties.level_population import (
    LevelNumberDensity,
)


class LTELevelNumberDensity(LevelNumberDensity):
    outputs = ('lte_level_number_density',)
    latex_name = ('N_{i,j,k}^*',)

    def _calculate_dilute_lte(self, lte_level_boltzmann_factor_Te, lte_ion_number_density,
                              levels, lte_partition_function_Te):
        return super()._calculate_dilute_lte(
            lte_level_boltzmann_factor_Te, lte_ion_number_density, levels, lte_partition_function_Te)

In [8]:
lte_ion_number_density = LTEIonNumberDensity(plasma,
                                             electron_densities=plasma.electron_densities
                                             ).calculate(
                                                 plasma.thermal_phi_lte,
                                                 plasma.thermal_lte_partition_function,
                                                 sim_state.elemental_number_density,
                                                 plasma.electron_densities,
                                                 None, 1e-20)[0]

In [9]:
lte_level_number_density = LTELevelNumberDensity(plasma).calculate(
                                                     plasma.thermal_lte_level_boltzmann_factor,
                                                     lte_ion_number_density,
                                                     plasma.levels,
                                                     plasma.thermal_lte_partition_function)

# Initialize ionization solvers

In [10]:
from tardis.plasma.equilibrium.rate_matrix import IonRateMatrix
from tardis.plasma.equilibrium.rates import (
    AnalyticPhotoionizationRateSolver,
    CollisionalIonizationRateSolver,
)

photoionization_rate_solver = AnalyticPhotoionizationRateSolver(atom_data2.photoionization_data)

collisional_rate_solver = CollisionalIonizationRateSolver(atom_data2.photoionization_data)

ion_rate_matrix_solver = IonRateMatrix(photoionization_rate_solver, collisional_rate_solver)

## Optional: solve for radiative ionization rates

In [11]:
photoion_rates, recomb_rates = photoionization_rate_solver.solve(
    rad_field,
    electron_dist,
    lte_level_number_density.loc[lte_level_number_density.index.get_level_values('ion_number') < 1],
    plasma.level_number_density.loc[plasma.level_number_density.index.get_level_values('ion_number') < 1],
    lte_ion_number_density.loc[lte_ion_number_density.index.get_level_values('ion_number') >= 1],
    plasma.ion_number_density.loc[plasma.ion_number_density.index.get_level_values('ion_number') >= 1],)

## Optional: solve for collisional ionization rates

In [12]:
saha_factor = lte_level_number_density.loc[lte_level_number_density.index.get_level_values('ion_number') < 1] / (
            lte_ion_number_density.loc[lte_ion_number_density.index.get_level_values('ion_number') >= 1].values
            * electron_dist.number_density.value
        )

coll_ionization_rate, coll_recomb_rate = collisional_rate_solver.solve(
    electron_dist,
    saha_factor,
)

In [13]:
saha_factor

0             1   \
atomic_number ion_number level_number                               
1             0          0             2.954267e-15  2.954267e-15   
                         1             8.561027e-20  8.561027e-20   
                         2             2.152074e-20  2.152074e-20   
                         3             1.776595e-20  1.776595e-20   
                         4             1.946301e-20  1.946301e-20   
                         5             2.311055e-20  2.311055e-20   
                         6             2.800258e-20  2.800258e-20   
                         7             3.391578e-20  3.391578e-20   
                         8             4.075987e-20  4.075987e-20   
                         9             4.849225e-20  4.849225e-20   
                         10            5.709047e-20  5.709047e-20   
                         11            6.654177e-20  6.654177e-20   
                         12            7.683840e-20  7.683840e-20   
                         13            8.797545e-20  8.797545e-20   
                         14            9.994965e-20  9.994965e-20   
                         15            1.127588e-19  1.127588e-19   
                         16            1.264013e-19  1.264013e-19   
                         17            1.408760e-19  1.408760e-19   
                         18            1.561821e-19  1.561821e-19   
                         19            1.723189e-19  1.723189e-19   
                         20            1.892861e-19  1.892861e-19   
                         21            2.070832e-19  2.070832e-19   
                         22            2.257099e-19  2.257099e-19   
                         23            2.451661e-19  2.451661e-19   
                         24            2.654514e-19  2.654514e-19   
                         25            2.865659e-19  2.865659e-19   
                         26            3.085093e-19  3.085093e-19   
                         27            3.312816e-19  3.312816e-19   
                         28            3.548827e-19  3.548827e-19   
                         29            3.793125e-19  3.793125e-19   

                                                 2             3   \
atomic_number ion_number level_number                               
1             0          0             2.954267e-15  2.954267e-15   
                         1             8.561027e-20  8.561027e-20   
                         2             2.152074e-20  2.152074e-20   
                         3             1.776595e-20  1.776595e-20   
                         4             1.946301e-20  1.946301e-20   
                         5             2.311055e-20  2.311055e-20   
                         6             2.800258e-20  2.800258e-20   
                         7             3.391578e-20  3.391578e-20   
                         8             4.075987e-20  4.075987e-20   
                         9             4.849225e-20  4.849225e-20   
                         10            5.709047e-20  5.709047e-20   
                         11            6.654177e-20  6.654177e-20   
                         12            7.683840e-20  7.683840e-20   
                         13            8.797545e-20  8.797545e-20   
                         14            9.994965e-20  9.994965e-20   
                         15            1.127588e-19  1.127588e-19   
                         16            1.264013e-19  1.264013e-19   
                         17            1.408760e-19  1.408760e-19   
                         18            1.561821e-19  1.561821e-19   
                         19            1.723189e-19  1.723189e-19   
                         20            1.892861e-19  1.892861e-19   
                         21            2.070832e-19  2.070832e-19   
                         22            2.257099e-19  2.257099e-19   
                         23            2.451661e-19  2.451661e-19   
                         24            2.65451

## Optional: solve for the ionization rate matrix

In [ ]:
ion_rate_matrix = ion_rate_matrix_solver.solve(
    rad_field,
    electron_dist,
    lte_level_number_density.loc[lte_level_number_density.index.get_level_values('ion_number') < 1],
    plasma.level_number_density.loc[plasma.level_number_density.index.get_level_values('ion_number') < 1],
    lte_ion_number_density.loc[lte_ion_number_density.index.get_level_values('ion_number') >= 1],
    plasma.ion_number_density.loc[plasma.ion_number_density.index.get_level_values('ion_number') >= 1],
    charge_conservation=True)

Ionization rate matrix for cell 0. First row is charge conservation. Second row is number conservation.

In [ ]:
ion_rate_matrix[0].values

# Solve ion population

In [ ]:
from tardis.plasma.equilibrium.ion_populations import IonPopulationSolver

solver = IonPopulationSolver(ion_rate_matrix_solver)

# Charge conservation is currently required to be True for the solver to work correctly.

ion_pops, electron_densities = solver.solve(
    rad_field,
    electron_dist,
    lte_level_number_density,
    plasma.level_number_density,
    lte_ion_number_density,
    plasma.ion_number_density,
    charge_conservation=True)

In [ ]:
ion_pops

In [ ]:
electron_densities